# MiniCPM聊天机器人
MiniCPM3-4B 是 MiniCPM 系列的第三代模型，具备40亿参数，它使用了包含大规模中英文语料的训练数据，支持 32k 长度上下文，并通过 LLMxMapReduce 技术实现理论上的无限上下文处理。MiniCPM 3.0，相比 MiniCPM1.0/2.0，功能更加全面，综合能力大幅提升，多数评测集上的效果比肩甚至超越众多 7B-9B 模型。

在本次任务中，我们将基于昇思MindSpore在香橙派开发板上运行MiniCPM3-4B模型，体验和模型的对话互动，完成和聊天机器人对话。具体操作如下：

## 环境准备

开发者拿到香橙派开发板后，首先需要进行硬件资源确认，镜像烧录及CANN和MindSpore版本的升级，才可运行该案例，具体如下：

- 硬件： 香橙派AIpro 24G 20T开发板
- 镜像： 香橙派官网ubuntu镜像
- CANN：8.0.0.beta1
- MindSpore： 2.5.0

### 镜像烧录

运行该案例需要烧录香橙派官网ubuntu镜像，烧录流程参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--镜像烧录](https://www.mindspore.cn/docs/zh-CN/r2.5.0/orange_pi/environment_setup.html#1-%E9%95%9C%E5%83%8F%E7%83%A7%E5%BD%95%E4%BB%A5windows%E7%B3%BB%E7%BB%9F%E4%B8%BA%E4%BE%8B)章节。

### CANN升级

CANN升级参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--CANN升级](https://www.mindspore.cn/docs/zh-CN/r2.5.0/orange_pi/environment_setup.html#3-cann%E5%8D%87%E7%BA%A7)章节。

### MindSpore升级

MindSpore升级参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--MindSpore升级](https://www.mindspore.cn/docs/zh-CN/r2.5.0/orange_pi/environment_setup.html#4-mindspore%E5%8D%87%E7%BA%A7)章节。

## 权重加载

此处使用mindnlp套件加载MindSpore-Lab/MiniCPM3-4B-FP16模型权重，该套件包含了许多自然语言处理的常用方法，可以方便地加载和使用modelers的模型权重。注意，首次运行时请耐心等待模型下载。

In [ ]:
#install mindnlp

!pip install git+https://github.com/mindspore-lab/mindnlp.git@0.4

%env TE_PARALLEL_COMPILER=1
%env MAX_COMPILE_CORE_NUMBER=1

In [ ]:
import mindspore
from mindnlp.transformers import MiniCPM3ForCausalLM, MiniCPM3Tokenizer
from mindspore._c_expression import disable_multi_thread

disable_multi_thread()
# Loading the tokenizer and model from Modelers's model hub.
model_id = "MindSpore-Lab/MiniCPM3-4B-FP16"
tokenizer = MiniCPM3Tokenizer.from_pretrained(model_id, mirror='modelers', ms_dtype=mindspore.float16)
model = MiniCPM3ForCausalLM.from_pretrained(model_id, mirror='modelers', ms_dtype=mindspore.float16, low_cpu_mem_usage=True)

## 模型推理

模型加载完成后定义聊天机器人所需处理的事务：加载与处理聊天历史，转换成适合模型输入的格式；并通过流式生成器的推理模式逐步生成聊天机器人的回复。

In [2]:
from mindnlp.transformers import TextIteratorStreamer
from threading import Thread

system_prompt = "You are a helpful and friendly chatbot"

def build_input_from_chat_history(chat_history, msg: str):
    messages = [{'role': 'system', 'content': system_prompt}]
    for user_msg, ai_msg in chat_history:
        messages.append({'role': 'user', 'content': user_msg})
        messages.append({'role': 'assistant', 'content': ai_msg})
    messages.append({'role': 'user', 'content': msg})
    return messages

# Function to generate model predictions.
def predict(message, history):
    # Formatting the input for the model.
    messages = build_input_from_chat_history(history, message)
    input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="ms",
            tokenizer=True
        )
    streamer = TextIteratorStreamer(tokenizer, timeout=180, skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        input_ids=input_ids,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.7,
        temperature=0.7,
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()  # Starting the generation in a separate thread.
    partial_message = ""
    for new_token in streamer:
        partial_message += new_token
        if '</s>' in partial_message:  # Breaking the loop if the stop token is generated.
            break
        yield partial_message

## 启动聊天机器人

下面我们将启动一个基于Gradio的聊天界面，用于与聊天机器人进行交互。在浏览器中打开链接 [http://127.0.0.1:7860](http://127.0.0.1:7860)，开始与聊天机器人的交互。您可以在页面下方的消息输入框中输入任何问题，或者点击页面下方 **Examples** 中预设的问题，然后点击 **Submit** 按钮与聊天机器人进行对话。注意，第一次回答需要较长时间的加载（约1~2分钟），请耐心等待。

In [ ]:
#install gradio

!pip install gradio==4.44.0

In [3]:
import gradio as gr
# Setting up the Gradio chat interface.
gr.ChatInterface(predict,
                 title="MiniCPM3",
                 description="问几个问题",
                 examples=['推荐5个北京的景点。']
                 ).launch()  # Launching the web interface.

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/home/HwHiAiUser/.local/lib/python3.9/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


本案例已同步上线 [GitHub 仓](https://github.com/mindspore-courses/orange-pi-mindspore/tree/master/Online/19-MiniCPM3)，更多案例开发亦可参考该仓库。

本案例运行所需环境：

- **硬件**：香橙派 AIpro 24G 20T 开发板
- **镜像**：香橙派官网 Ubuntu 镜像
- **CANN**：8.0.0.beta1
- **MindSpore**：2.5.0